In [1]:
import pandas as pd

import numpy as np

In [2]:
train=pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
data = pd.concat([train,test])
data.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [3]:
data.describe()
#可见数值型数据中Age与Fare有缺值

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,1309.000000,891.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000
mean,655.000000,0.383838,2.294882,29.881138,0.498854,0.385027,33.295479
std,378.020061,0.486592,0.837836,14.413493,1.041658,0.865560,51.758668
min,1.000000,0.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,328.000000,0.000000,2.000000,21.000000,0.000000,0.000000,7.895800
50%,655.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,982.000000,1.000000,3.000000,39.000000,1.000000,0.000000,31.275000
max,1309.000000,1.000000,3.000000,80.000000,8.000000,9.000000,512.329200


In [4]:
data['Age']=data['Age'].fillna(data['Age'].median())  #中位值填充
data['Fare']=data['Fare'].fillna(data['Fare'].median())

print(data['Sex'].unique())  #看str行有几类
data.loc[data['Sex']=='male','Sex']=0  #基于标签的索引loc(数据编号（行索引），数据特征值（列索引）)
data.loc[data['Sex']=='female','Sex']=1

print(data['Embarked'].unique())
#观察到Embarked也有缺值
data['Embarked']=data['Embarked'].fillna('S')  #众数填充
data.loc[data['Embarked']=='S','Embarked']=0
data.loc[data['Embarked']=='C','Embarked']=1
data.loc[data['Embarked']=='Q','Embarked']=2

#无用特征删除
data=data.drop('Name',axis=1)
data=data.drop('Ticket',axis=1)
data=data.drop('Cabin',axis=1)

#训练集，测试集分离
train = data.loc[data["PassengerId"]<892]
test = data.loc[data["PassengerId"]>891]
data = train
data=data.drop('PassengerId',axis=1)
test=test.drop('PassengerId',axis=1)
test=test.drop('Survived',axis=1)

['male' 'female']
['S' 'C' 'Q' nan]


In [5]:
data.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.361582,0.523008,0.381594,32.204208
std,0.486592,0.836071,13.019697,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [6]:
data.head(3)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0.0,3,0,22.0,1,0,7.2500,0
1,1.0,1,1,38.0,1,0,71.2833,1
2,1.0,3,1,26.0,0,0,7.9250,0


In [7]:
test.head(3)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0,34.5,0,0,7.8292,2
1,3,1,47.0,1,0,7.0000,0
2,2,0,62.0,0,0,9.6875,2


In [8]:
test.describe()

,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,418.000000,418.000000,418.000000
mean,2.265550,29.805024,0.447368,0.392344,35.576535
std,0.841838,12.667969,0.896760,0.981429,55.850103
min,1.000000,0.170000,0.000000,0.000000,0.000000
25%,1.000000,23.000000,0.000000,0.000000,7.895800
50%,3.000000,28.000000,0.000000,0.000000,14.454200
75%,3.000000,35.750000,1.000000,0.000000,31.471875
max,3.000000,76.000000,8.000000,9.000000,512.329200


In [9]:
def corr(x,y):
    x=x.to_numpy()
    y=y.to_numpy()
    cov = (x*y).mean()-x.mean()*y.mean()
    #std函数默认参数计算样本两阶中心距
    return cov/(x.std()*y.std())
C = np.zeros((7,7))
for i,key1 in enumerate(data.columns[1:]):
    for j,key2 in enumerate(data.columns[1:]):
        C[i,j]=corr(data[key1],data[key2])
        if key1 == key2:
            continue
        print("corr(%s,%s) = %.3f"%(key1,key2,corr(data[key1],data[key2])))

corr(Pclass,Sex) = -0.132
corr(Pclass,Age) = -0.340
corr(Pclass,SibSp) = 0.083
corr(Pclass,Parch) = 0.018
corr(Pclass,Fare) = -0.549
corr(Pclass,Embarked) = 0.046
corr(Sex,Pclass) = -0.132
corr(Sex,Age) = -0.081
corr(Sex,SibSp) = 0.115
corr(Sex,Parch) = 0.245
corr(Sex,Fare) = 0.182
corr(Sex,Embarked) = 0.117
corr(Age,Pclass) = -0.340
corr(Age,Sex) = -0.081
corr(Age,SibSp) = -0.233
corr(Age,Parch) = -0.172
corr(Age,Fare) = 0.097
corr(Age,Embarked) = -0.009
corr(SibSp,Pclass) = 0.083
corr(SibSp,Sex) = 0.115
corr(SibSp,Age) = -0.233
corr(SibSp,Parch) = 0.415
corr(SibSp,Fare) = 0.160
corr(SibSp,Embarked) = -0.060
corr(Parch,Pclass) = 0.018
corr(Parch,Sex) = 0.245
corr(Parch,Age) = -0.172
corr(Parch,SibSp) = 0.415
corr(Parch,Fare) = 0.216
corr(Parch,Embarked) = -0.079
corr(Fare,Pclass) = -0.549
corr(Fare,Sex) = 0.182
corr(Fare,Age) = 0.097
corr(Fare,SibSp) = 0.160
corr(Fare,Parch) = 0.216
corr(Fare,Embarked) = 0.062
corr(Embarked,Pclass) = 0.046
corr(Embarked,Sex) = 0.117
corr(Embarked,Age)

In [10]:
for i in range(7):
    for j in range(i):
        if abs(C[i,j]) >= 0.4:
            print("filter %s and %s their corr is %.3f"%(data.columns[i+1],data.columns[j+1],C[i,j]))

filter Parch and SibSp their corr is 0.415
filter Fare and Pclass their corr is -0.549


In [11]:
#两者的离散值数量相同
print(data["Parch"].unique())
print(data["SibSp"].unique())

[0 1 2 5 3 4 6]
[1 0 3 4 2 5 8]


In [12]:
data = data.drop(["Parch","Fare"],axis=1)
test = test.drop(["Parch","Fare"],axis=1)


In [13]:
data.head()

,Survived,Pclass,Sex,Age,SibSp,Embarked
0,0.0,3,0,22.0,1,0
1,1.0,1,1,38.0,1,1
2,1.0,3,1,26.0,0,0
3,1.0,1,1,35.0,1,0
4,0.0,3,0,35.0,0,0


In [14]:
test.head()

,Pclass,Sex,Age,SibSp,Embarked
0,3,0,34.5,0,2
1,3,1,47.0,1,0
2,2,0,62.0,0,2
3,3,0,27.0,0,0
4,3,1,22.0,1,0


In [15]:
def frequency(data,feature,x,y):
    #特征取值数量
    m = len(data[feature].unique())
    alpha = 1
    #N_{a_i}
    N_y = len(data.loc[data["Survived"]==y,feature])
    #N_{a_i}^{b_i}
    N_x = len(data.loc[data["Survived"]==y,feature].loc[data[feature]==x])
    return (N_x+alpha)/(N_y+m*alpha)
def gauss(data,feature,x,y):
    #计算高斯分布
    mean = np.mean(data.loc[data["Survived"]==y,feature].to_numpy())
    #numpy默认两阶中心矩
    var = np.var(data.loc[data["Survived"]==y,feature].to_numpy())
    return np.exp(-(x-mean)**2/(2*var))/np.sqrt(2*np.pi*var)
def bayes(data,test,output = True):
    pred = np.zeros(len(test),dtype=np.int)
    for i in range(test.shape[0]):
        #先验概率，由于是两点分布，采用直接的频率估计，无需拉普拉斯平滑
        P_0 = len(data.loc[data["Survived"]==0,"Survived"])/len(data["Survived"])
        P_1 = len(data.loc[data["Survived"]==1,"Survived"])/len(data["Survived"])
        results = np.array([P_0,P_1])
        sample = test.iloc[i]
        for y in range(2):
            for feature in test.columns:
                if(feature == "Age"):  
                    #连续型
                    results[y]*=gauss(data,str(feature),sample[feature],y)
                    continue
                #离散型
                results[y]*=frequency(data,str(feature),sample[feature],y)
        #归一化
        results=results/np.sum(results)
        pred[i] = np.argmax(results)
        if output:
            print("Sample%d's results are"%i,results,"pred is%d"%pred[i])
    return pred

In [16]:
pred=bayes(data,test.iloc[:5])

Sample0's results are [0.9159649 0.0840351] pred is0
Sample1's results are [0.33266876 0.66733124] pred is1
Sample2's results are [0.74170999 0.25829001] pred is0
Sample3's results are [0.92784997 0.07215003] pred is0
Sample4's results are [0.30981175 0.69018825] pred is1


In [17]:
yp = bayes(data,test,False)

In [18]:
passenger_id = pd.read_csv('test.csv')["PassengerId"]

predict=np.stack((passenger_id,yp),axis=1)

index=['PassengerId','Survived']
pre=pd.DataFrame(predict,columns=index)  #将其保存为dataframe

pre.to_csv('my_submission_p.csv',index = False)  #保存为csv文件

In [19]:
pre

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
